# Named Entity Recognition without labelled data: A weak supervision approach

In this walkthrough, we will develop a neural NER model without access to labelled data. 

**Important note**: some of the data/model files used in this walkthrough are too big to be put on the GitHub repository, but are accessible for download [here](https://github.com/NorskRegnesentral/skweak/releases/tag/0.2.8).

Let's look at a particular example of text (from Reuters):


In [2]:
import sys
sys.path.insert(0, '../..')


In [6]:
import re
news_text  = """
ATLANTA  (Reuters) - Retailer Best Buy Co, seeking new ways to appeal to cost-conscious shoppers, said on Tuesday it is selling refurbished 
 versions of Apple Inc's iPhone 3G at its stores that are priced about $50 less than new iPhones. 
 The electronics chain said the used iPhones, which were returned within 30 days of purchase, are priced at $149 for the model with 8 gigabytes of storage, 
 while the 16-gigabyte version is $249. A two-year service contract with AT&T Inc is required. New iPhone 3Gs currently sell for $199 and $299 at 
 Best Buy Mobile stores. "This is focusing on customers' needs, trying to provide as wide a range of products and networks for our consumers," said 
 Scott Moore, vice president of marketing for Best Buy Mobile. Buyers of first-generation iPhones can also upgrade to the faster refurbished 3G models at 
 Best Buy, he said. Moore said AT&T, the exclusive wireless provider for the iPhone, offers refurbished iPhones online. The sale of used iPhones comes as 
 Best Buy, the top consumer electronics chain, seeks ways to fend off increased competition from discounters such as Wal-Mart Stores Inc, which began 
 selling the popular phone late last month. Wal-Mart sells a new 8-gigabyte iPhone 3G for $197 and $297 for the 16-gigabyte model. The iPhone is also 
 sold at Apple stores and AT&T stores. Moore said Best Buy's move was not in response to other retailers' actions. (Reporting by  Karen Jacobs ; Editing 
 by  Andre Grenon )"""

news_text = re.sub('\\s+', ' ', news_text)

Let's view what the standard Spacy model produces (note it takes a few seconds to reload the vocabulary):

In [7]:
import spacy, skweak

# We load the spacy model
nlp = spacy.load("en_core_web_sm")
doc = nlp(news_text)

# Visualising the entities
skweak.utils.display_entities(doc)

The medium-size model works better, but still contains quite a few errors and omissions:

In [4]:

# We load the spacy model (takes a few seconds)
nlp = spacy.load("en_core_web_md")
doc = nlp(news_text)

# Visualising the entities
skweak.utils.display_entities(doc)

Ideally, one would wish to train a better named entity recognition model, which is better tailored to the specific needs and linguistic patterns found in those articles. However, although we have large amounts of raw text data, we often do not have text data labelled with named entities for our domain. We therefore worked on an alternative approach based on __weak supervision__, combining several (noisy) supervision sources instead of a single "gold standard". 

Indeed, we do have access to several possible supervision sources, such as alternative NER models trained on other corpora, large lists of entity (companies, person names, geographical locations), shallow linguistic patterns, and document-level constraints. 

The key idea behind the proposed approach is thus to (1) use these supervision sources to automatically annotate news corpora, (2) estimate a label model (more precisely an HMM model) that unifies all these sources into a single one, and (3) learn a new NER model based on these unified labels. <br>

__Outline of this notebook__: I describe below the various annotation schemes that I developed.  I then explain how these various sources can be merged into a single source using the `skweak` framework. Finally, I detail the architecture behind the NER model.

## __Step 1:__ Annotations

### 1) Annotators from other Spacy models

A first source of automatic annotation comes from NER models trained on multiple, distinct corpora. I went through [available NE-labelled corpora](https://github.com/juand-r/entity-recognition-datasets) to search for datasets that could be used to train alternative models. I then trained Spacy models for all of them, and then conducted some experiments to assess their performance. At the end of the process, I ended up with four models:
- The standard Spacy model for English (`en_core_web_md`), trained on Ontonotes v5
- A model trained on [ConLL 2003](https://www.clips.uantwerpen.be/conll2003/ner/)
- A model trained on the [Broad Twitter Corpus](https://github.com/GateNLP/broad_twitter_corpus)
- A model trained on a corpus of [SEC filings](https://www.aclweb.org/anthology/U15-1010/).
    
Note there are differences between the entity labels of these models: while Ontonotes contains no less than [18 classes](https://spacy.io/api/annotation#named-entities), the other corpora only contain `PER(SON)`, `ORG`, `LOC` and `MISC`. Furthermore, the labels also do not match each other perfectly: while Ontonotes distinguishes between geopolitical locations (`GPE`) and "natural" locations (such as continents, seas etc., labelled as `LOC`), the three other models regroup all geographical entities as `LOC`. 

We can apply annotations from a Spacy model using the `ModelAnnotator` class.

In [15]:
annotator = skweak.spacy.ModelAnnotator("conll2003", "../../data/conll2003/")

doc = annotator(doc)
skweak.utils.display_entities(doc, "conll2003")

As we can see, the results are not perfect on this model either, but the errors are distinct from the ones made by the Ontonotes model. 

The annotations are written in the `spans` of the Spacy document:

In [11]:
[(ent, ent.label_) for ent in doc.spans["conll2003"]]

[(ATLANTA, 'ORG'),
 (Reuters, 'ORG'),
 (Best Buy Co, 'ORG'),
 (Apple Inc, 'ORG'),
 (iPhone, 'ORG'),
 (iPhones, 'ORG'),
 (iPhones, 'ORG'),
 (AT&T Inc, 'ORG'),
 (New iPhone, 'LOC'),
 (Best Buy Mobile, 'LOC'),
 (Scott Moore, 'PER'),
 (Best Buy Mobile, 'ORG'),
 (iPhones, 'ORG'),
 (Best Buy, 'LOC'),
 (Moore, 'PER'),
 (AT&T, 'PER'),
 (iPhone, 'LOC'),
 (iPhones, 'ORG'),
 (iPhones, 'ORG'),
 (Best Buy, 'LOC'),
 (Wal, 'LOC'),
 (Mart Stores Inc, 'ORG'),
 (Wal, 'LOC'),
 (Mart, 'ORG'),
 (iPhone, 'ORG'),
 (iPhone, 'LOC'),
 (Apple, 'LOC'),
 (Moore, 'PER'),
 (Best Buy, 'ORG'),
 (Karen Jacobs, 'PER'),
 (Andre Grenon, 'PER')]

Each `ModelAnnotator` adds two annotation sources: one that is directly based on the Spacy Model (here `conll2003`), and one that also includes the corrections specified in the method `_correct_entities` (in `spacy_wrapper.py`) that we implemented earlier this year.  The corrected version are indicated with a `+c` suffix.

Here are the results from the three other models:

In [17]:
core_web_annotator = skweak.spacy.ModelAnnotator("core_web_md", "en_core_web_md")

doc = core_web_annotator(doc)
skweak.utils.display_entities(doc, "core_web_md")


__Note__: When annotating large collections of news documents, the method `annotator.pipe(news_docs)` is much more efficient than calling `annotate(...)` every single time, as it batches the documents on which to run the NER model.

### 2) Annotators from gazetteers

Another useful source of annotation comes from large lists of entities such as persons, places and organisations. The gazetteers are using a _trie_ to efficiently search for occurrences in the text. Gazetteers can be run in two modes: case-sensitive or case-insentitive.


#### 2.1) Wikipedia
The database from Wikipedia is extracted from the [NECKar](https://event.ifi.uni-heidelberg.de/?page_id=532) dataset.  The postprocessing (which, among others, filters out entities that are also relatively common English words) is implemented in `compile_wikidata` as [WIKIDATA](https://github.com/NorskRegnesentral/skweak/releases/download/v0.2/wikidata_tokenised.json.gz). In addition, I also extracted from Wikidata a list of commercial products and added them to the gazetteer. 

In [18]:
import examples.ner.conll2003_ner

tries = skweak.gazetteers.extract_json_data("../../data/wikidata_tokenised.json")
annotator = skweak.gazetteers.GazetteerAnnotator("wiki", tries)

annotator(doc)
skweak.utils.display_entities(doc, "wiki")

Extracting data from ../../data/wikidata_tokenised.json
Populating trie for class PERSON (number: 2621131)
Populating trie for class LOC (number: 47104)
Populating trie for class GPE (number: 601419)
Populating trie for class ORG (number: 295449)
Populating trie for class PRODUCT (number: 12457)


Again, the annotation model does make some errors: `Moore` is thought to be a [geopolitical entity](https://en.wikipedia.org/wiki/Moore) instead of a person. Note that `AT&T` has two alternative labels: `ORG` or `GPE` (see [AT&T station](https://en.wikipedia.org/wiki/AT%26T_(SEPTA_station))). The data is available as [WIKI_SMALL](https://github.com/NorskRegnesentral/skweak/raw/main/data/wikidata_small_tokenised.json.gz).

In addition to the full wiki data, I also added a specific gazetteer that only employs wikidata objects containing a text description:

In [19]:
tries = skweak.gazetteers.extract_json_data("../../data/wikidata_small_tokenised.json")
annotator = skweak.gazetteers.GazetteerAnnotator("wikismall_cased", tries)
annotator2 = skweak.gazetteers.GazetteerAnnotator("wikismall_uncased", tries, case_sensitive=False)

annotator2(annotator(doc))
skweak.utils.display_entities(doc, "wikismall_cased")
print()
skweak.utils.display_entities(doc, "wikismall_uncased")


Extracting data from ../../data/wikidata_small_tokenised.json
Populating trie for class PERSON (number: 1863434)
Populating trie for class LOC (number: 14241)
Populating trie for class GPE (number: 273373)
Populating trie for class ORG (number: 91341)
Populating trie for class PRODUCT (number: 12457)


As we can see, the "cased" gazetteers have a higher precision than the uncased gazetteers (at a cost of lower coverage).

#### 2.2 Crunchbase

The second gazetteer [Crunchbase](https://github.com/NorskRegnesentral/skweak/raw/main/data/crunchbase_companies.json.gz) is extracted from the [Open Data Map from Crunchbase](https://data.crunchbase.com/docs/open-data-map), which contains lists of both organisations and (business) persons.

In [20]:
tries = skweak.gazetteers.extract_json_data("../../data/crunchbase.json",  spacy_model="en_core_web_sm")
annotator = skweak.gazetteers.GazetteerAnnotator("crunchbase_cased", tries)
annotator2 = skweak.gazetteers.GazetteerAnnotator("crunchbase_uncased", tries)

annotator2(annotator(doc))
skweak.utils.display_entities(doc, ["crunchbase_cased", "crunchbase_uncased"])

Extracting data from ../../data/crunchbase.json
Populating trie for class COMPANY (number: 788714)
Populating trie for class ORG (number: 261)
Populating trie for class PERSON (number: 1062669)


#### 2.3 Geonames

The [geonames](http:www.geonames.org) database [GEO_NAMES](https://github.com/NorskRegnesentral/skweak/blob/main/data/geonames.json) contains a large list of locations, including both geopolitical entities and "natural" locations:

In [21]:
tries = skweak.gazetteers.extract_json_data("../../data/geonames.json",  spacy_model="en_core_web_sm")
annotator = skweak.gazetteers.GazetteerAnnotator("geo_cased", tries)
annotator2 = skweak.gazetteers.GazetteerAnnotator("geo_uncased", tries, case_sensitive=False)

annotator2(annotator(doc))
skweak.utils.display_entities(doc, ["geo_cased", "geo_uncased"])

Extracting data from ../../data/geonames.json
Populating trie for class GPE (number: 15205)


#### 2.4 Product names

Finally, I used [DBPedia](http://www.dbpedia.org) to extract a list of products and brands as [Products](https://github.com/NorskRegnesentral/skweak/blob/main/data/products.json), since the recognition of products is particularly poor in Spacy NER models:

In [22]:

tries = skweak.gazetteers.extract_json_data("../../data/products.json",  spacy_model="en_core_web_sm")
annotator = skweak.gazetteers.GazetteerAnnotator("products_cased", tries)
annotator2 = skweak.gazetteers.GazetteerAnnotator("products_uncased", tries)

annotator2(annotator(doc))
skweak.utils.display_entities(doc, ["products_cased", "products_uncased"])

Extracting data from ../../data/products.json
Populating trie for class PRODUCT (number: 45362)


### 3. Shallow patterns

Some named entities can also be captured through relatively simple, handcrafted patterns defined on the Spacy document. The class `FunctionAnnotator` makes it easy to define an annotator based on a function that takes a Spacy document as input and generate text spans with a label. Relations of mutual exclusivity between annotation sources can also be specified in the annotator. For instance, we can specify that numbers that are part of a date, time or money span should be ignored from the "number_detector" (to avoid having e.g. the `21` in `October 21` labelled as a `CARDINAL`): 

In [23]:
import examples.ner
date_annotator = skweak.heuristics.FunctionAnnotator("date_detector", examples.ner.conll2003_ner.date_generator)
time_annotator = skweak.heuristics.FunctionAnnotator("time_detector", examples.ner.conll2003_ner.time_generator)
money_annotator = skweak.heuristics.FunctionAnnotator("money_detector", examples.ner.conll2003_ner.money_generator)
exclusives = ["date_detector", "time_detector", "money_detector"]
number_annotator = skweak.heuristics.FunctionAnnotator("number_detector", examples.ner.conll2003_ner.number_generator)
number_annotator.add_incompatible_sources(exclusives)

date_annotator(doc)
time_annotator(doc)
money_annotator(doc)
number_annotator(doc)
skweak.utils.display_entities(doc, "date_detector")
skweak.utils.display_entities(doc, "time_detector")
skweak.utils.display_entities(doc, "money_detector")
skweak.utils.display_entities(doc, "number_detector")

I have also created a range of patterns aiming to improve the _detection_ of named entities, even though they leave the actual label underspecified (as a generic `ENT` label). Four such detectors are constructed:
- two detectors of proper names based on casing (marking sequence of tokens whose lemma are "titled" as potential named entities)
- one detector of NNP sequences (based on the Spacy POS tagger)
- and one detector of sequences with proper names linked with "compound" dependency relations

In [10]:
# Detection based on casing
proper_detector = skweak.heuristics.TokenConstraintAnnotator("proper_detector", skweak.utils.is_likely_proper, "ENT")
    
# Detection based on casing, but allowing some lowercased tokens
proper2_detector = skweak.heuristics.TokenConstraintAnnotator("proper2_detector", skweak.utils.is_likely_proper, "ENT")
proper2_detector.add_gap_tokens(examples.ner.data_utils.LOWERCASED_TOKENS | examples.ner.data_utils.NAME_PREFIXES)
#add  .ner.        
# Detection based on part-of-speech tags
nnp_detector = skweak.heuristics.TokenConstraintAnnotator("nnp_detector", lambda tok: tok.tag_=="NNP", "ENT")
        
# Detection based on dependency relations (compound phrases)
compound = lambda tok: skweak.utils.is_likely_proper(tok) and skweak.utils.in_compound(tok)
compound_detector = skweak.heuristics.TokenConstraintAnnotator("compound_detector", compound, "ENT")
 
combined = skweak.base.CombinedAnnotator()
exclusives = ["date_detector", "time_detector", "money_detector"]
for annotator in [proper_detector, proper2_detector, nnp_detector, compound_detector]:
    annotator.add_incompatible_sources(exclusives)
    annotator.add_gap_tokens(["'s", "-"])
    combined.add_annotator(annotator)

    # We add one variants for each NE detector, looking at infrequent tokens
    infrequent_name = "infrequent_%s"%annotator.name
    combined.add_annotator(skweak.heuristics.SpanConstraintAnnotator(infrequent_name, annotator.name, skweak.utils.is_infrequent))

doc = combined(doc)
skweak.utils.display_entities(doc, "proper_detector")
skweak.utils.display_entities(doc, "proper2_detector")
skweak.utils.display_entities(doc, "nnp_detector")
skweak.utils.display_entities(doc, "compound_detector")

Finally, I created three specific annotators:
- to recognise company names with a legal type
- full person names (with a first name along a list of common first names)
- slightly less common entities such as `NORP`, `FAC`, `LANGUAGE`, `EVENT` and `LAW`.

In [11]:

# Other types (legal references etc.)      
misc_detector = skweak.heuristics.FunctionAnnotator("misc_detector", examples.ner.conll2003_ner.misc_generator)
legal_detector = skweak.heuristics.FunctionAnnotator("legal_detector", examples.ner.conll2003_ner.legal_generator)
        
# Detection of companies with a legal type
ends_with_legal_suffix = lambda x: x[-1].lower_.rstrip(".") in examples.ner.data_utils.LEGAL_SUFFIXES
company_type_detector = skweak.heuristics.SpanConstraintAnnotator("company_type_detector", "proper2_detector", 
                                                    ends_with_legal_suffix, "COMPANY")
# Detection of full person names
FIRSR_NAMES = "../../data/first_names.json"
full_name_detector = skweak.heuristics.SpanConstraintAnnotator("full_name_detector", "proper2_detector", 
                                                     examples.ner.conll2003_ner.FullNameDetector(), "PERSON")


legal_detector(doc)
company_type_detector(doc)
full_name_detector(doc)
misc_detector(doc)
skweak.utils.display_entities(doc, "company_type_detector")
skweak.utils.display_entities(doc, "full_name_detector")
skweak.utils.display_entities(doc, "misc_detector")
skweak.utils.display_entities(doc, "legal_detector")

Finally, we also rely on an external probabilistic [parser of named entities](https://github.com/snipsco/snips-nlu-parsers) from [Snips](https://snips.ai/). The parser recognises `DATE`, `TIME`, `ORDINAL`, `CARDINAL`, `MONEY` and `PERCENT`. The parser is implemented in _Rust_, so it runs quite fast.

In [12]:
# Detection based on a probabilistic parser
# NB: requires to install "snips-nlu-parsers" (pip install snips-nlu-parsers)
snips = examples.ner.conll2003_ner.SnipsAnnotator("snips")
snips(doc)
skweak.utils.display_entities(doc, "snips")

### 4. Document-level annotators

All annotators presented so far rely on _local_ decisions on tokens or phrases.  However, news articles are not mere collections of words, but exhibit a high degree of internal coherence. This can be exploited to furhter improve the annotation. Two document-level annotators are implemented:

Before we can run the document-level annotators, we need to normalise some of the entities. The `ConLL2003Standardiser` is responsible for this normalisation:
- entities `PER` (from conll2003, BTC and SEC) are set to `PERSON`
- entities `LOC` from conll2003, BTC and SEC for spans that are also annotated by other layers as `GPE` are set to `GPE` 
- entities `ORG` that are annotated by other layers as `COMPANY` are set to `COMPANY`
    

In [26]:
annotator = examples.ner.conll2003_ner.ConLL2003Standardiser()
doc = annotator(doc)

In [27]:
mv = skweak.aggregation.MajorityVoter("mv", ["LOC", "MISC", "ORG", "PER"])
mv.add_underspecified_label("ENT", {"LOC", "MISC", "ORG", "PER"})
doc = mv(doc)
skweak.utils.display_entities(doc, "mv")

#### 4.1 Document history

When a journalist first mentions an entity such as a company or person in an article, they typically write it in a "long form", and then use shorter mentions once the entity is properly introduced. For instance, in the text above, "Scott Moore" is first mentioned with a full name, and then simply referred to as "Moore". Similarly, companies are often first introduced to with their legal type.  The `DocumentHistoryAnnotator` takes advantage of this property, by propagating the label from the first mention onto subsequent mentions:

In [28]:
annotator = skweak.doclevel.DocumentHistoryAnnotator("doc_history", "mv", ["PER", "ORG"])
annotator(doc)
skweak.utils.display_entities(doc, "doc_history")

#### 4.2 Label consistency

Another property of news documents is the fact that two (or more) named entities sharing the same string in a text typically refer to the same entity, and should therefore have the same label. "Komatsu" can be both a company name and a city in Japan, but within a given document, it will typically be one or the other for the whole document. We can capture this fact with an annotator that looks at the majority label for a given string, and annotate all occurrences with this label:

In [29]:
annotator = skweak.doclevel.DocumentMajorityAnnotator("doc_majority", "mv")
annotator(doc)
skweak.utils.display_entities(doc, "doc_majority")

In [30]:
mv = skweak.aggregation.MajorityVoter("mv", ["LOC", "MISC", "ORG", "PER"])
mv.add_underspecified_label("ENT", {"LOC", "MISC", "ORG", "PER"})
doc = mv(doc)
skweak.utils.display_entities(doc, "mv")

## __Step 2__: Estimation of label model

We can construct a full annotator with all annotators described above, and then run it on a dataset such as Reuters, Bloomberg, or Acquire:

In [31]:
full_annotator = examples.ner.conll2003_ner.NERAnnotator().add_all()
print("Total number of annotators:", len(full_annotator.annotators))

Loading shallow functions
Loading Spacy NER models
Loading gazetteer supervision modules
Extracting data from ../../domains/ner/../../data/wikidata_tokenised.json
Populating trie for class PERSON (number: 2621131)
Populating trie for class LOC (number: 47104)
Populating trie for class GPE (number: 601419)
Populating trie for class ORG (number: 295449)
Populating trie for class PRODUCT (number: 12457)
Extracting data from ../../domains/ner/../../data/wikidata_small_tokenised.json
Populating trie for class PERSON (number: 1863434)
Populating trie for class LOC (number: 14241)
Populating trie for class GPE (number: 273373)
Populating trie for class ORG (number: 91341)
Populating trie for class PRODUCT (number: 12457)
Extracting data from ../../domains/ner/../../data/geonames.json
Populating trie for class GPE (number: 15205)
Extracting data from ../../domains/ner/../../data/crunchbase.json
Populating trie for class COMPANY (number: 788714)
Populating trie for class ORG (number: 261)
Popul

We can then take the raw data from [Reuters](https://github.com/NorskRegnesentral/skweak/raw/main/data/reuters_small.tar.gz), run Spacy on the textual content, and finally apply the annotator to get annotations from the each source:

In [34]:
# We annotate 200 documents, and store them in a Spacy DocBin file
docs = list(skweak.utils.docbin_reader("../../data/reuters_small.spacy"))
docs list(full_annotator.pipe(docs))

Once this is done, we can finally estimate a unified annotator model through weak supervision. The basic idea is to describe the named entity recognition problem as a _Hidden markov Model_ where the observations are the annotations from each source, and the states correspond to the "true" (hidden) labels for each token, as illustrated in Figure 2 in the paper.

Since we don't have access to the true labels for each token, we will rely on _Baum-Welch_ (a variant of EM) to estimate the HMM model through unsupervised training. More specifically, we will need to estimate 3 models:
- the initial probabilities $P(Y_0)$ of the labels for the first token of a document
- the transition matrix $P(Y_i | Y_{i-1})$ for the labels 
- the emission models $P(\lambda_{i,j} | Y_i)$ of observing a particular value $\lambda_{i,j}$ (say, `B-PER`) from the source $j$ given the true label $Y_i$. In the current model, we assume the emissions to be independent of one another given the true label, to reduce the complexity of the model.

Given an annotated dataset, the HMM model can be easily estimated:

In [42]:
unified_model = skweak.aggregation.HMM("hmm", ["LOC", "MISC", "ORG", "PER"])
unified_model.add_underspecified_label("ENT", ["LOC", "MISC", "ORG", "PER"])
# We then run Baum-Welch on the model (can take some time)
unified_model.fit(docs)

# Saving the model to a file
unified_model.save("../../data/hmm_reuters_small.pkl")

Starting iteration 1
Finished E-step with 195 documents
Starting iteration 2
         1     -358446.4555             +nan
Finished E-step with 195 documents
Starting iteration 3
         2     -347323.9735      +11122.4821
Finished E-step with 195 documents
Starting iteration 4
         3     -346892.7074        +431.2661
Finished E-step with 195 documents
         4     -346777.5166        +115.1908


Note that the HMM model relies on some informative priors to facilitate the parameter estimation:
- the prior for the initial probabilities is a Dirichlet based on counts for the most reliable model (chosen right now to be the Spacy NER model trained on Ontonotes)
- the prior for the transition matrix is a list of Dirichlet also based on counts from the standard Spacy NER model.
- finally, the initial emission models are calculated based on subjective estimates of the relative precision and recall of each source. For instance, we know that a source like `company_type_detector` (which looks at legal suffixes such as "Inc." at the end of the noun phrase) has a very high precision, but a low recall , since many mentions of companies do not include a suffix. In contrast, gazeteers will tend to have a better recall, but a lower precision (some company names also happen to be names of geopolitical entities or persons).  The initial precisions and recalls provided to the model are specified in `SOURCE_PRIORS` in the file `labelling.py`. When a precision and recall is not provided for a given source, they are assumed to be zeros (for instance, `company_type_detector` only detects `COMPANY` entities and nothing else).  

In [40]:
#unified_model.pretty_print() 

Once the model is learned, we can apply it as any other "annotator" object:

In [41]:
docs = list(unified_model.pipe(docs))
skweak.utils.display_entities(docs[0], "hmm")
skweak.utils.display_entities(docs[1], "hmm")

<br>

## __Step 3__: Development of neural NER model


We can now learn a neural NER model based on these unified annotations. We have two options: a straighforward (but slightly underperforming) approach using Spacy, and a more sophisticated approach using our own NER model

### __Alternative 1__: Using Spacy

In [43]:
for doc in docs:
    doc.ents = doc.spans["hmm"]
skweak.utils.docbin_writer(docs, "../../data/reuters_small.spacy")

Write to ../../data/reuters_small.spacy...done


And we can then directly train a new NER model with Spacy's training regime:

In [44]:
!spacy init config - --lang en --pipeline ner --optimize accuracy | \
spacy train - --paths.train ../../data/reuters_small.spacy  --paths.dev ../../data/reuters_small.spacy \
--initialize.vectors en_core_web_md --output ../../data/reuters_small


2021-04-19 11:13:04.422733: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-04-19 11:13:04.443580: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-04-19 11:13:11,975] [INFO] Set up nlp object from config
[2021-04-19 11:13:11,985] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-04-19 11:13:11,989] [INFO] Created vocabulary
[2021-04-19 11:13:14,790] [INFO] Added vectors: en_core_web_md
[2021-04-19 11:13:14,790] [INFO] Finished initializing nlp object
^C

Aborted!


We then load the learned model (taking the model version with best performance on the development set):

In [39]:
nlp = spacy.load("../data/reuters_small_spacy/model-best")

And we finally run it on the document:

In [40]:
skweak.utils.display_entities(nlp(news_text))

NB: The file `eval_utils.py` contains code to easily extract evaluation metrics by comparing the annotations from a particular annotation layer (for instance the HMM predictions, or the predictions from a single source) to the gold standard: